In [6]:
import torch
import torch.nn as nn
import torch.utils.data
#assert(torch.cuda.is_available()) # if this fails go to Runtime -> Change runtime type -> Set "Hardware Accelerator"
print("Torch version:", torch.__version__)
import torch.nn.functional as F
import cv2
import matplotlib.pyplot as plt
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import transforms

Torch version: 1.11.0


In [14]:
cat_to_numeric={}
with open("family.label.conversion.txt") as f:
    for line in f:
      #  print(line)
        cat_to_numeric[line.split()[0]]= line.split()[-1]

In [15]:
isolate = []
family = []
with open ("all.data") as f:
    for line in f:
        isolate.append (line.split()[0])
        family.append(line.split()[1])

In [4]:
all_data=[]
for i in range(len(isolate)):
    iso=isolate[i]
    lab= int(cat_to_numeric[family[i]])
    #print(iso)
    img_path='sub_images_final/'+iso+'.trans.bmp'
    bgr_img = cv2.imread(img_path)
    # Convert to grayscale
    gray_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY)
    gray_img = gray_img.astype("float32")/255
    # Plot image
    #plt.imshow(bgr_img, cmap='gray')
    #plt.show()
    gray_img_tensor = torch.from_numpy(gray_img).unsqueeze(0)
    gray_img_tensor=(gray_img_tensor,lab)
    all_data.append(gray_img_tensor)

In [27]:
all_data=[]
for i in range(len(isolate)):
    iso=isolate[i]
    lab= int(cat_to_numeric[family[i]])
    #print(iso)
    img_path='sub_images_final/'+iso+'.trans.bmp'
    input_image = Image.open(img_path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
    input_tensor = preprocess(input_image)
    
   # input_batch = input_tensor.unsqueeze(0)
    input_batch_tensor=(input_tensor,lab)
    all_data.append(input_batch_tensor)

In [28]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(all_data, test_size=0.2)

In [29]:
train_label=[]

for i in range(len(train_data)):
    train_label.append(train_data[i][1])
print("number of training classes are:",len(set(train_label)) )

test_label=[]
for i in range(len(test_data)):
    test_label.append(test_data[i][1])
print("number of test classes are:",len(set(test_label)) )

number of training classes are: 16
number of test classes are: 16


In [32]:
train_loader = DataLoader(train_data, batch_size=20, shuffle=True, num_workers=0)
test_loader= DataLoader(test_data,  shuffle=True, num_workers=0)

In [36]:
train_data[0][0].shape

torch.Size([3, 224, 224])

In [24]:
input_batch.shape

torch.Size([1, 3, 224, 224])

In [37]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)



Using cache found in /Users/yiwei/.cache/torch/hub/pytorch_vision_v0.10.0


In [38]:
from torch.optim import Adam
 
# Define the loss function with Classification Cross-Entropy loss and an optimizer with Adam optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.000001, weight_decay=0.0001)

In [ ]:
with torch.no_grad():
    output = model(input_batch)

In [41]:
from torch.autograd import Variable

# Function to save the model
def saveModel():
    path = "./ResNet18Model.pth"
    torch.save(model.state_dict(), path)

# Function to test the model with the test dataset and print the accuracy for the test images
def testAccuracy():
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
          #  print(labels)
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)


# Training function. We simply have to loop over our data iterator and feed the inputs to the network and optimize.
def train(num_epochs):
    
    best_accuracy = 0.0

    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        running_acc = 0.0

        for i, (images, labels) in enumerate(train_loader, 0):
           # print(labels)
            # get the inputs
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))
            #print(labels)
            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(images)
            #print(outputs)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, labels)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

            # Let's print statistics for every 1,000 images
            #print(i)
            running_loss += loss.item()     # extract the loss value
            if i % 500 == 499:    
                # print every 1000 (twice per epoch) 
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 500))
                # zero the loss
                running_loss = 0.0

        # Compute and print the average accuracy fo this epoch when tested over all 10000 test images
        accuracy = testAccuracy()
        print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
        
        # we want to save the model if the accuracy is the best
        if accuracy > best_accuracy:
            saveModel()
            best_accuracy = accuracy

In [43]:
train(10)

The model will be running on cpu device
[1,   500] loss: 6.607
For epoch 1 the test accuracy over the whole test set is 27 %
[2,   500] loss: 2.429
For epoch 2 the test accuracy over the whole test set is 40 %
[3,   500] loss: 1.891
For epoch 3 the test accuracy over the whole test set is 37 %
[4,   500] loss: 1.849
For epoch 4 the test accuracy over the whole test set is 40 %
[5,   500] loss: 1.831
For epoch 5 the test accuracy over the whole test set is 40 %
[6,   500] loss: 1.817
For epoch 6 the test accuracy over the whole test set is 41 %
[7,   500] loss: 1.804
For epoch 7 the test accuracy over the whole test set is 40 %
[8,   500] loss: 1.788
For epoch 8 the test accuracy over the whole test set is 41 %
[9,   500] loss: 1.784
For epoch 9 the test accuracy over the whole test set is 41 %
[10,   500] loss: 1.765
For epoch 10 the test accuracy over the whole test set is 39 %


In [46]:
path = "ResNet18Model.pth"
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [47]:
predicted_label=[]
def testAccuracy():
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
          #  print(labels)
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            predicted_label.append(int(predicted[0]))
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)

In [48]:
accuracy=testAccuracy()

In [51]:
print(accuracy)

41.452991452991455


In [49]:
results={}
for i in set(test_label):
    #print(i)
    #true positive:
    tp=0
    fp=0
    tn=0
    fn=0
    for c in range(len(predicted_label)):
        if predicted_label[c] == i:
            if test_label[c]==i:
                tp+=1
            else:
                fp+=1
        else:
            if test_label[c]!=i:
                tn+=1
            else:
                fn+=1
    prec=0
    rec=0
    if (tp+fp) >0:
        prec= tp/(tp+fp)
    if (tp+fn)>0:
        rec=tp/(tp+fn)
    results[i]=[tp, fp, tn, fn, prec,rec]

In [50]:
results

{0: [754, 1274, 824, 424, 0.3717948717948718, 0.6400679117147708],
 1: [294, 877, 1636, 469, 0.251067463706234, 0.3853211009174312],
 2: [0, 0, 3054, 222, 0, 0.0],
 3: [6, 14, 3059, 197, 0.3, 0.029556650246305417],
 4: [0, 0, 3108, 168, 0, 0.0],
 5: [0, 0, 3135, 141, 0, 0.0],
 6: [0, 0, 3177, 99, 0, 0.0],
 7: [0, 2, 3187, 87, 0.0, 0.0],
 8: [0, 0, 3191, 85, 0, 0.0],
 9: [0, 8, 3174, 94, 0.0, 0.0],
 10: [0, 0, 3225, 51, 0, 0.0],
 11: [0, 0, 3237, 39, 0, 0.0],
 12: [0, 0, 3263, 13, 0, 0.0],
 13: [0, 1, 3264, 11, 0.0, 0.0],
 14: [0, 0, 3263, 13, 0, 0.0],
 15: [1, 45, 3122, 108, 0.021739130434782608, 0.009174311926605505]}